In [39]:
import pandas as pd
import urllib2 

In [48]:
# TODO: download from URL
dropbox = "/Users/mr/Dropbox/moviemeta/"

In [49]:
with open(dropbox + "imdb/genres.list") as f:
    genres = f.readlines()
with open(dropbox + "imdb/plot.list") as f:
    plots = f.readlines()
with open(dropbox + "imdb/countries.list") as f:
    countries = f.readlines()

In [50]:
print len(genres)
print len(plots)
print len(countries)

2134860
6855909
1713923


In [8]:
plotlist = []
nextMovie = False
plot = []
revplots = []
nextPlot = True
by = False
for line in plots:
    #The titles that start with double quotes are TV shows, exclude them
    if line[:2] == 'MV' and line[4] == '"':
        nextMovie = False
        continue
    if line[:2] == 'MV':
        title = line[4:-1]
        i = title.find('(')
        j = title.find(')')
        if i > 0 and j > 0:
            year = title[i+1:j]
        nextMovie = True
    if nextMovie == True:
        revplots = []
        #instantiate a dict for the next movie
        movie = {'title':title, 'plots':revplots, 'year':year, 'type':'user plot', 'source':'imdb'}
        plotlist.append(movie)
        nextMovie = False
    if nextPlot == True:
        #instantiate a new plot
        plot = []
    if line[:2] == 'PL':
        #append to plot
        plot.append(line[4:].replace('\n',' '))
        nextPlot = False
        by = False
    if line[:2] == 'BY':
        #we're at the end of a plot, convert to string and append to list of plots for this review
        plotstr = ''.join(plot)
        revplots.append(plotstr.strip()) 
        nextPlot = True
        by = True
    if line[:2] == '--':
        if by == False:  #if there was no 'BY' line we append now
            plotstr = ''.join(plot)
            revplots.append(plotstr.strip())  
        nextPlot = True
        by = False
    

In [59]:
movieplots = pd.DataFrame(data=plotlist, columns=['title','source','year','plots', 'type'])

In [60]:
movieplots.head()

,title,source,year,plots,type
0,#1 Cheerleader Camp (2010) (V),imdb,2010,[When they're hired to work at a cheerleading ...,user plot
1,#1 Serial Killer (2013),imdb,2013,[Years of seething rage against the racism he'...,user plot
2,#1 at the Apocalypse Box Office (2015),imdb,2015,"[Jules is, self declared, the most useless per...",user plot
3,#137 (2011),imdb,2011,"[#137 is a SCI/FI thriller about a girl, Marla...",user plot
4,#29 (2012),imdb,2012,"[In #29, the constant zooming into certain lan...",user plot


In [28]:
'''#this takes forever, make it more efficient
for movie in countries:
    if movie[0] == '"':
        continue
    split = movie.split('\t')
    title = split[0]
    idx = movieplots[movieplots['title']== title].index
    #if the title is in our dataframe add the country
    if len(idx > 0):
        movieplots.loc[[idx[0]],['country']]= split[len(split)-1].replace('\n','')
'''

In [62]:
movieplots.to_csv(dropbox + "movieplots.csv", index=False)